In [5]:
import os
import tensorflow as tf
os.chdir('../')

In [6]:
os.getcwd()

'/Users/vikaslakka/Desktop/FSDS/DeepLearning/Projects/CNN_projects/First_classifier'

In [7]:
from firstClassifier.constants import *
from firstClassifier.utils.common import read_yaml, create_directories

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PreapareCallbacksConfig:
  root_dir: Path
  tensorboard_root_log_dir: Path
  checkpoint_model_filepath: Path

In [20]:
class ConfigurationManager:

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH
             ):
        """_summary_: Creates artifact directories taken from config.yaml file
        """
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        ##Create directories
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)->PreapareCallbacksConfig:
        """Preparing callback configs

        Returns:
            PreapareCallbackConfig: PreapareCallbackConfig
        """
        ##Config contains the attributes that are present in prepare_base_model in config.yaml
        
        config= self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        ##Create directory for checkpoint, tensorboard root log
        create_directories([Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)
                            ])

        prepare_callback_config= PreapareCallbacksConfig(   
                root_dir= config.root_dir,
                tensorboard_root_log_dir= config.tensorboard_root_log_dir,
                checkpoint_model_filepath= config.checkpoint_model_filepath

            )
        return prepare_callback_config

In [21]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
class PrepareCallback:
    """Get the base model and update the parameters with dataset.
    Picking up Defined architectures in Tensorflow and updating with dataset available
    """
    def __init__(self, config: PreapareCallbacksConfig):
        self.config= config

    @property
    def _create_tb_callbacks(self):
        timestamp= time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir= os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    

In [22]:
try:
    config= ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(prepare_callbacks_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e


[2023-03-30 18:05:41,088: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2023-03-30 18:05:41,090: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-30 18:05:41,091: INFO: common]: created directory at: artifacts
[2023-03-30 18:05:41,091: INFO: common]: created directory at: artifacts/prepare_callbacks/checkpoint_dir
[2023-03-30 18:05:41,092: INFO: common]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
